# 2: Train XGBoost Model

Author: Daniel Lusk

## Imports and configuration

In [11]:
from TrainModelConfig import TrainModelConfig
from utils.data_retrieval import all_gdfs

%load_ext autoreload
%autoreload 2

config = TrainModelConfig()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [12]:
X_fns = config.WC_fns + config.MODIS_fns + config.soil_fns
y_fns = config.iNat_fns

X = all_gdfs(X_fns)
y = all_gdfs(y_fns)

Compute Preciptation Annual Range by subtracting BIO14 from BIO13

In [15]:
bio_13 = X.loc[:, ["bio_13" in x for x in X.columns]].values
bio_14 = X.loc[:, ["bio_14" in x for x in X.columns]].values
X["wc2.1_10m_bio_13-14"] = bio_13 - bio_14

Drop the unnecessary `band`, `spatial_ref`, and `geometry` columns.

In [22]:
X = X.drop(columns=["band", "spatial_ref", "geometry"])
y = y.drop(columns=["band", "spatial_ref", "geometry"])

## XGBoost